In [1]:
import subprocess
import csv

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager

import IPython.display
import ipywebrtc
import pydub

chandas = matplotlib.font_manager.FontProperties(fname="chandas.ttf", size=18)

average_male_ipa = [
    ["i", 240, 2400],
    ["y", 235, 2100],
    ["e", 390, 2300],
    ["ø", 370, 1900],
    ["ɛ", 610, 1900],
    ["œ", 585, 1710],
    ["a", 850, 1610],
    ["ɶ", 820, 1530],
    ["ɑ", 750, 940],
    ["ɒ", 700, 760],
    ["ʌ", 600, 1170],
    ["ɔ", 500, 700],
    ["ɤ", 460, 1310],
    ["o", 360, 640],
    ["ɯ", 300, 1390],
    ["u", 250, 595],
]

devanagari = [
    ["अ/a", 630.174, 1226.89],   # sounds/a.wav
    ["आ/ā", 964.683, 1423.33],   # sounds/aa.wav
    ["इ/i", 548.374, 2094.91],   # sounds/i.wav
    ["ई/ī", 354.247, 2461.84],   # sounds/ii.wav
    ["उ/u", 569.609, 1458.78],   # sounds/u.wav
    ["ऊ/ū", 477.939, 1284.99],   # sounds/oo.wav
    ["ऋ/ṛ", 611.441, 2047.6],   # sounds/ri.wav
    ["ए/e", 441.785, 2578.6],   # sounds/e.wav
    ["ऐ/ai", 741.582, 2032.28],   # sounds/ai.wav
    ["ओ/o", 564.162, 1322.18],   # sounds/o.wav
    ["औ/au", 730.529, 1523.9],   # sounds/au.wav
]

def number(x):
    try:
        return float(x)
    except ValueError:
        return float("nan")

def plot(recorder, alphabet, font_prop=None, tmp_filename="/tmp/praat-formants-sample"):
    recorder.save(tmp_filename + ".webm")

    webm = pydub.AudioSegment.from_file(tmp_filename + ".webm")
    webm.export(tmp_filename + ".wav", format="wav")

    formants = np.array(
        [
            list(map(number, x))
            for x in csv.reader(
                subprocess.run(
                    [
                        "praat_nogui", "--run", "formants.praat", tmp_filename + ".wav", "5500", "0.025", "50"
                    ],
                    stdout=subprocess.PIPE,
                )
                .stdout.decode()
                .strip()
                .split("\n"),
                delimiter="\t",
            )
        ][1:]
    )
    intensity = np.array(
        [
            list(map(number, x))
            for x in csv.reader(
                subprocess.run(
                    [
                        "praat_nogui", "--run", "intensity.praat", tmp_filename + ".wav"
                    ],
                    stdout=subprocess.PIPE,
                )
                .stdout.decode()
                .strip()
                .split("\n"),
                delimiter=" ",
            )
        ]
    )
    intensity = intensity[2:2 + len(formants)]
    assert np.max(formants[:, 0] - intensity[:, 0]) < 0.004

    no_nan = ~(np.isnan(formants[:, 1]) | np.isnan(formants[:, 2]) | np.isnan(intensity[:, 1]))
    weights = (intensity[:, 1] - np.nanmin(intensity[:, 1]))[no_nan]

    fig, ax = plt.subplots(figsize=(6, 6), dpi=120)
    ax.hist2d(
        x=formants[no_nan, 1],
        y=formants[no_nan, 2],
        weights=weights,
        bins=(100, 100),
        range=((100, 1100), (250, 3000)),
        density=True,
        vmin=1e-5,
        vmax=1e-4,
        cmap="Oranges",
    );
    for vowel, x, y in alphabet:
        ax.text(x, y, vowel, va="center", ha="center", fontsize="x-large", c="blue", fontproperties=font_prop);
    ax.set_xlabel("$F_1$ (Hz)");
    ax.set_ylabel("$F_2$ (Hz)");

camera = ipywebrtc.CameraStream(constraints={"audio": True, "video": False})
recorder = ipywebrtc.AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [ ]:
plot(recorder, average_male_ipa)

In [ ]:
plot(recorder, devanagari, font_prop=chandas)

[hindilanguage.info: Devanagari Vowels](https://hindilanguage.info/lessons/lesson-2-devanagari-vowels/)

In [2]:
#    अ   /   a
IPython.display.Audio("sounds/a.wav")

In [3]:
#    आ   /   ā
IPython.display.Audio("sounds/aa.wav")

In [4]:
#    इ   /   i
IPython.display.Audio("sounds/i.wav")

In [5]:
#    ई   /   ī
IPython.display.Audio("sounds/ii.wav")

In [6]:
#    उ   /   u
IPython.display.Audio("sounds/u.wav")

In [7]:
#    ऊ   /   ū
IPython.display.Audio("sounds/oo.wav")

In [8]:
#    ऋ   /   ṛ
IPython.display.Audio("sounds/ri.wav")

In [9]:
#    ए   /   e
IPython.display.Audio("sounds/e.wav")

In [10]:
#    ऐ   /   ai
IPython.display.Audio("sounds/ai.wav")

In [11]:
#    ओ   /   o
IPython.display.Audio("sounds/o.wav")

In [12]:
#    औ   /   au
IPython.display.Audio("sounds/au.wav")